<a href="https://colab.research.google.com/github/hugow0528/AI/blob/main/tax_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#version 0

In [ ]:
from collections import deque
import time

# 模擬用戶數據庫
users_db = {}

# 用戶登錄歷史堆疊
login_history = deque(maxlen=10)

def calculate_progressive_tax(taxable_income):
    """計算累進稅率的稅款"""
    if taxable_income <= 0:
        return 0

    tax = 0
    """對應薪俸稅的累進稅率表的前四級"""
    brackets = [(50000, 0.02), (50000, 0.06), (50000, 0.10), (50000, 0.14)]
    remaining = taxable_income

    for bracket_limit, rate in brackets:
        if remaining > bracket_limit:
            tax += bracket_limit * rate
            remaining -= bracket_limit
        else:
            tax += remaining * rate
            remaining = 0
            break

    if remaining > 0:
        tax += remaining * 0.17

    return tax

def calculate_standard_tax(net_income):
    """計算標準稅率的稅款"""
    if net_income <= 0:
        return 0
    return net_income * 0.15

def calculate_hk_salary_tax(annual_income, deductions, allowances):
    """計算香港薪俸稅"""
    net_income = annual_income - deductions
    taxable_income = max(0, net_income - allowances)
    progressive_tax = calculate_progressive_tax(taxable_income)
    standard_tax = calculate_standard_tax(net_income)
    final_tax = min(progressive_tax, standard_tax)
    final_tax_after_reduction = max(0, final_tax - 1500)

    return {
        '總收入': annual_income,
        '扣除額': deductions,
        '免稅額': allowances,
        '入息淨額': net_income,
        '應課稅入息': taxable_income,
        '累進稅率稅款': progressive_tax,
        '標準稅率稅款': standard_tax,
        '最終稅款(未減免)': final_tax,
        '最終稅款(減免後)': final_tax_after_reduction
    }

def get_user_input():
    """獲取用戶輸入並計算總扣除額和免稅額"""
    print("\n香港2024/25年度薪俸稅計算機")
    print("=================================")

    try:
        income = float(input("請輸入年薪 (港元): "))

        print("\n可扣稅項目 (輸入0表示無相關項目):")
        mpf = min(float(input("強積金供款總額 (上限18,000港元): ")), 18000)
        charity = float(input("認可慈善捐款總額: "))
        elderly_care = min(float(input("長者住宿照顧開支 (上限100,000港元): ")), 100000)
        home_loan_interest = min(float(input("居所貸款利息 (上限100,000港元): ")), 100000)
        vhps = float(input("自願醫保計劃保費 (每名受保人上限8,000港元): "))
        annuity_mpf_voluntary = min(float(input("合資格年金保費及自願強積金供款 (合計上限60,000港元): ")), 60000)
        rent = min(float(input("住宅租金扣除 (上限100,000港元): ")), 100000)
        fertility = min(float(input("輔助生育服務支出 (上限100,000港元): ")), 100000)

        other_deductions = mpf + elderly_care + home_loan_interest + vhps + annuity_mpf_voluntary + rent + fertility
        charity_max = (income - other_deductions) * 0.35
        charity = min(charity, charity_max)

        total_deductions = mpf + charity + elderly_care + home_loan_interest + vhps + annuity_mpf_voluntary + rent + fertility

        print("\n免稅額項目:")
        is_married = input("是否已婚? (y/n): ").lower() == 'y'
        total_allowances = 264000 if is_married else 132000

        num_children = int(input("子女數量: "))
        total_allowances += num_children * 130000

        num_siblings = int(input("供養兄弟姊妹數量: "))
        total_allowances += num_siblings * 37500

        parents_55_59 = int(input("供養55-59歲父母/祖父母數量: "))
        parents_60_plus = int(input("供養60歲或以上父母/祖父母數量: "))
        parents_disabled = int(input("供養傷殘父母/祖父母數量 (未滿60歲但符合傷殘津貼資格): "))
        total_allowances += (parents_55_59 * 25000 + parents_60_plus * 50000 + parents_disabled * 50000)

        is_single_parent = input("是否單親? (y/n): ").lower() == 'y'
        if is_single_parent:
            total_allowances += 132000

        disabled_dependents = int(input("傷殘受養人數量: "))
        total_allowances += disabled_dependents * 75000

        is_disabled = input("您是否傷殘人士? (y/n): ").lower() == 'y'
        if is_disabled:
            total_allowances += 75000

        return income, total_deductions, total_allowances

    except ValueError:
        print("請輸入有效的數字！")
        return None, None, None
    except Exception as e:
        print(f"發生錯誤: {e}")
        return None, None, None


"""用戶註冊功能"""
def register():
    print("\n=== 用戶註冊 ===")
    username = input("請輸入用戶名: ").strip()
    if username in users_db:
        print("用戶名已存在，請選擇其他用戶名！")
        return False

    password = input("請輸入密碼: ").strip()
    users_db[username] = {'password': password}
    print("註冊成功！請登入使用。")
    return True



"""用戶登入功能"""
def login():
    print("\n=== 用戶登入 ===")
    username = input("請輸入用戶名: ").strip()
    password = input("請輸入密碼: ").strip()

    if username in users_db and users_db[username]['password'] == password:
        print(f"歡迎回來，{username}！")
        login_history.append(f"{username} logged in at {time.strftime('%Y-%m-%d %H:%M:%S')}")
        return username
    else:
        print("用戶名或密碼錯誤！")
        return None


"""顯示登錄歷史"""
def show_login_history():
    print("\n=== 最近登錄歷史 ===")
    if not login_history:
        print("尚無登錄記錄。")
    else:
        for entry in reversed(login_history):
            print(entry)

def main():
    while True:
        print("\n=== 香港薪俸稅計算系統 ===")
        print("1. 註冊")
        print("2. 登入")
        print("3. 查看登錄歷史")
        print("4. 退出")

        choice = input("請選擇操作 (1-4): ").strip()

        if choice == '1':
            register()

        elif choice == '2':
            username = login()
            if username:
                income, deductions, allowances = get_user_input()
                if income is not None:
                    result = calculate_hk_salary_tax(income, deductions, allowances)
                    print("\n計算結果:")
                    print("----------------")
                    for key, value in result.items():
                        print(f"{key}: ${value:,.2f}")

        elif choice == '3':
            show_login_history()

        elif choice == '4':
            print("感謝使用，再見！")
            break

        else:
            print("無效選項，請重新選擇！")

if __name__ == "__main__":
    main()


=== 香港薪俸稅計算系統 ===
1. 註冊
2. 登入
3. 查看登錄歷史
4. 退出
請選擇操作 (1-4): 43
無效選項，請重新選擇！

=== 香港薪俸稅計算系統 ===
1. 註冊
2. 登入
3. 查看登錄歷史
4. 退出
請選擇操作 (1-4): 4
感謝使用，再見！


# version 1 database login

In [ ]:
"""version1"""

from collections import deque
import time
import json
import os

# 用戶資料檔案路徑
USER_DB_FILE = "users_db.json"

# 用戶登錄歷史堆疊
login_history = deque(maxlen=10)

# 載入用戶資料並確保每個用戶有 tax_history
def load_users_db():
    """從檔案載入用戶資料，並為舊用戶補上 tax_history"""
    if os.path.exists(USER_DB_FILE):
        with open(USER_DB_FILE, 'r', encoding='utf-8') as f:
            db = json.load(f)
            # 為每個用戶補上 tax_history（如果缺失）
            for username in db:
                if 'tax_history' not in db[username]:
                    db[username]['tax_history'] = []
            return db
    return {}

# 保存用戶資料
def save_users_db(users_db):
    """將用戶資料保存到檔案"""
    with open(USER_DB_FILE, 'w', encoding='utf-8') as f:
        json.dump(users_db, f, ensure_ascii=False, indent=4)

# 初始化用戶數據庫
users_db = load_users_db()

def calculate_progressive_tax(taxable_income):
    """計算累進稅率的稅款"""
    if taxable_income <= 0:
        return 0

    tax = 0
    brackets = [(50000, 0.02), (50000, 0.06), (50000, 0.10), (50000, 0.14)]
    remaining = taxable_income

    for bracket_limit, rate in brackets:
        if remaining > bracket_limit:
            tax += bracket_limit * rate
            remaining -= bracket_limit
        else:
            tax += remaining * rate
            remaining = 0
            break

    if remaining > 0:
        tax += remaining * 0.17

    return tax

def calculate_standard_tax(net_income):
    """計算標準稅率的稅款"""
    if net_income <= 0:
        return 0
    return net_income * 0.15

def calculate_hk_salary_tax(annual_income, deductions, allowances):
    """計算香港薪俸稅"""
    net_income = annual_income - deductions
    taxable_income = max(0, net_income - allowances)
    progressive_tax = calculate_progressive_tax(taxable_income)
    standard_tax = calculate_standard_tax(net_income)
    final_tax = min(progressive_tax, standard_tax)
    final_tax_after_reduction = max(0, final_tax - 1500)

    return {
        '總收入': annual_income,
        '扣除額': deductions,
        '免稅額': allowances,
        '入息淨額': net_income,
        '應課稅入息': taxable_income,
        '累進稅率稅款': progressive_tax,
        '標準稅率稅款': standard_tax,
        '最終稅款(未減免)': final_tax,
        '最終稅款(減免後)': final_tax_after_reduction,
        '計算時間': time.strftime('%Y-%m-%d %H:%M:%S')
    }

def get_user_input():
    """獲取用戶輸入並計算總扣除額和免稅額"""
    print("\n香港2024/25年度薪俸稅計算機")
    print("=================================")

    try:
        income = float(input("請輸入年薪 (港元): "))

        print("\n可扣稅項目 (輸入0表示無相關項目):")
        mpf = min(float(input("強積金供款總額 (上限18,000港元): ")), 18000)
        charity = float(input("認可慈善捐款總額: "))
        elderly_care = min(float(input("長者住宿照顧開支 (上限100,000港元): ")), 100000)
        home_loan_interest = min(float(input("居所貸款利息 (上限100,000港元): ")), 100000)
        vhps = float(input("自願醫保計劃保費 (每名受保人上限8,000港元): "))
        annuity_mpf_voluntary = min(float(input("合資格年金保費及自願強積金供款 (合計上限60,000港元): ")), 60000)
        rent = min(float(input("住宅租金扣除 (上限100,000港元): ")), 100000)
        fertility = min(float(input("輔助生育服務支出 (上限100,000港元): ")), 100000)

        other_deductions = mpf + elderly_care + home_loan_interest + vhps + annuity_mpf_voluntary + rent + fertility
        charity_max = (income - other_deductions) * 0.35
        charity = min(charity, charity_max)

        total_deductions = mpf + charity + elderly_care + home_loan_interest + vhps + annuity_mpf_voluntary + rent + fertility

        print("\n免稅額項目:")
        is_married = input("是否已婚? (y/n): ").lower() == 'y'
        total_allowances = 264000 if is_married else 132000

        num_children = int(input("子女數量: "))
        total_allowances += num_children * 130000

        num_siblings = int(input("供養兄弟姊妹數量: "))
        total_allowances += num_siblings * 37500

        parents_55_59 = int(input("供養55-59歲父母/祖父母數量: "))
        parents_60_plus = int(input("供養60歲或以上父母/祖父母數量: "))
        parents_disabled = int(input("供養傷殘父母/祖父母數量 (未滿60歲但符合傷殘津貼資格): "))
        total_allowances += (parents_55_59 * 25000 + parents_60_plus * 50000 + parents_disabled * 50000)

        is_single_parent = input("是否單親? (y/n): ").lower() == 'y'
        if is_single_parent:
            total_allowances += 132000

        disabled_dependents = int(input("傷殘受養人數量: "))
        total_allowances += disabled_dependents * 75000

        is_disabled = input("您是否傷殘人士? (y/n): ").lower() == 'y'
        if is_disabled:
            total_allowances += 75000

        return income, total_deductions, total_allowances

    except ValueError:
        print("請輸入有效的數字！")
        return None, None, None
    except Exception as e:
        print(f"發生錯誤: {e}")
        return None, None, None

def register():
    """用戶註冊功能"""
    print("\n=== 用戶註冊 ===")
    username = input("請輸入用戶名: ").strip()
    if username in users_db:
        print("用戶名已存在，請選擇其他用戶名！")
        return False

    password = input("請輸入密碼: ").strip()
    users_db[username] = {'password': password, 'tax_history': []}
    save_users_db(users_db)
    print("註冊成功！請登入使用。")
    return True

def login():
    """用戶登入功能"""
    print("\n=== 用戶登入 ===")
    username = input("請輸入用戶名: ").strip()
    password = input("請輸入密碼: ").strip()

    if username in users_db and users_db[username]['password'] == password:
        print(f"歡迎回來，{username}！")
        login_history.append(f"{username} logged in at {time.strftime('%Y-%m-%d %H:%M:%S')}")
        return username
    else:
        print("用戶名或密碼錯誤！")
        return None

def show_login_history():
    """顯示登錄歷史"""
    print("\n=== 最近登錄歷史 ===")
    if not login_history:
        print("尚無登錄記錄。")
    else:
        for entry in reversed(login_history):
            print(entry)

def show_tax_history(username):
    """顯示用戶的稅務計算歷史"""
    print(f"\n=== {username} 的稅務計算歷史 ===")
    tax_history = users_db[username].get('tax_history', [])
    if not tax_history:
        print("尚無稅務計算記錄。")
    else:
        for i, record in enumerate(tax_history, 1):
            print(f"\n記錄 {i} ({record['計算時間']}):")
            print("----------------")
            for key, value in record.items():
                if key != '計算時間':
                    print(f"{key}: ${value:,.2f}")
                else:
                    print(f"{key}: {value}")

def main():
    while True:
        print("\n=== 香港薪俸稅計算系統 ===")
        print("1. 註冊")
        print("2. 登入")
        print("3. 查看登錄歷史")
        print("4. 退出")

        choice = input("請選擇操作 (1-4): ").strip()

        if choice == '1':
            register()

        elif choice == '2':
            username = login()
            if username:
                # 確保 tax_history 存在
                if 'tax_history' not in users_db[username]:
                    users_db[username]['tax_history'] = []
                    save_users_db(users_db)

                while True:
                    print(f"\n=== 歡迎，{username} ===")
                    print("1. 計算薪俸稅")
                    print("2. 查看稅務歷史")
                    print("3. 登出")
                    sub_choice = input("請選擇操作 (1-3): ").strip()

                    if sub_choice == '1':
                        income, deductions, allowances = get_user_input()
                        if income is not None:
                            result = calculate_hk_salary_tax(income, deductions, allowances)
                            print("\n計算結果:")
                            print("----------------")
                            for key, value in result.items():
                                if key != '計算時間':
                                    print(f"{key}: ${value:,.2f}")
                                else:
                                    print(f"{key}: {value}")
                            # 保存到用戶歷史
                            users_db[username]['tax_history'].append(result)
                            save_users_db(users_db)

                    elif sub_choice == '2':
                        show_tax_history(username)

                    elif sub_choice == '3':
                        print("已登出。")
                        break

                    else:
                        print("無效選項，請重新選擇！")

        elif choice == '3':
            show_login_history()

        elif choice == '4':
            print("感謝使用，再見！")
            break

        else:
            print("無效選項，請重新選擇！")

if __name__ == "__main__":
    main()

# version 2 with "system"
### final version
system account 可用於 :
1. 查看系統統計
2. 修改用戶憑證
3. 查看用戶稅務歷史

In [1]:
"""version 2"""

from collections import deque
import time
import json
import os
import matplotlib.pyplot as plt
import numpy as np

USER_DB_FILE = "users_db.json"
login_history = deque(maxlen=10)

def load_users_db():
    if os.path.exists(USER_DB_FILE):
        with open(USER_DB_FILE, 'r', encoding='utf-8') as f:
            db = json.load(f)
            for username in db:
                if 'tax_history' not in db[username]:
                    db[username]['tax_history'] = []
            return db
    return {"system": {"password": "system", "tax_history": []}}

def save_users_db(users_db):
    with open(USER_DB_FILE, 'w', encoding='utf-8') as f:
        json.dump(users_db, f, ensure_ascii=False, indent=4)

users_db = load_users_db()

def calculate_progressive_tax(taxable_income):
    if taxable_income <= 0:
        return 0
    tax = 0
    brackets = [(50000, 0.02), (50000, 0.06), (50000, 0.10), (50000, 0.14)]
    remaining = taxable_income
    for bracket_limit, rate in brackets:
        if remaining > bracket_limit:
            tax += bracket_limit * rate
            remaining -= bracket_limit
        else:
            tax += remaining * rate
            remaining = 0
            break
    if remaining > 0:
        tax += remaining * 0.17
    return tax

def calculate_standard_tax(net_income):
    if net_income <= 0:
        return 0
    return net_income * 0.15

def calculate_hk_salary_tax(annual_income, deductions, allowances):
    net_income = annual_income - deductions
    taxable_income = max(0, net_income - allowances)
    progressive_tax = calculate_progressive_tax(taxable_income)
    standard_tax = calculate_standard_tax(net_income)
    final_tax = min(progressive_tax, standard_tax)
    final_tax_after_reduction = max(0, final_tax - 1500)
    return {
        '總收入': annual_income,
        '扣除額': deductions,
        '免稅額': allowances,
        '入息淨額': net_income,
        '應課稅入息': taxable_income,
        '累進稅率稅款': progressive_tax,
        '標準稅率稅款': standard_tax,
        '最終稅款(未減免)': final_tax,
        '最終稅款(減免後)': final_tax_after_reduction,
        '計算時間': time.strftime('%Y-%m-%d %H:%M:%S')
    }

def get_user_input():
    print("\n香港2024/25年度薪俸稅計算機")
    print("=================================")
    try:
        income = float(input("請輸入年薪 (港元): "))
        print("\n可扣稅項目 (輸入0表示無相關項目):")
        mpf = min(float(input("強積金供款總額 (上限18,000港元): ")), 18000)
        charity = float(input("認可慈善捐款總額: "))
        elderly_care = min(float(input("長者住宿照顧開支 (上限100,000港元): ")), 100000)
        home_loan_interest = min(float(input("居所貸款利息 (上限100,000港元): ")), 100000)
        vhps = float(input("自願醫保計劃保費 (每名受保人上限8,000港元): "))
        annuity_mpf_voluntary = min(float(input("合資格年金保費及自願強積金供款 (合計上限60,000港元): ")), 60000)
        rent = min(float(input("住宅租金扣除 (上限100,000港元): ")), 100000)
        fertility = min(float(input("輔助生育服務支出 (上限100,000港元): ")), 100000)
        other_deductions = mpf + elderly_care + home_loan_interest + vhps + annuity_mpf_voluntary + rent + fertility
        charity_max = (income - other_deductions) * 0.35
        charity = min(charity, charity_max)
        total_deductions = mpf + charity + elderly_care + home_loan_interest + vhps + annuity_mpf_voluntary + rent + fertility
        print("\n免稅額項目:")
        is_married = input("是否已婚? (y/n): ").lower() == 'y'
        total_allowances = 264000 if is_married else 132000
        num_children = int(input("子女數量: "))
        total_allowances += num_children * 130000
        num_siblings = int(input("供養兄弟姊妹數量: "))
        total_allowances += num_siblings * 37500
        parents_55_59 = int(input("供養55-59歲父母/祖父母數量: "))
        parents_60_plus = int(input("供養60歲或以上父母/祖父母數量: "))
        parents_disabled = int(input("供養傷殘父母/祖父母數量 (未滿60歲但符合傷殘津貼資格): "))
        total_allowances += (parents_55_59 * 25000 + parents_60_plus * 50000 + parents_disabled * 50000)
        is_single_parent = input("是否單親? (y/n): ").lower() == 'y'
        if is_single_parent:
            total_allowances += 132000
        disabled_dependents = int(input("傷殘受養人數量: "))
        total_allowances += disabled_dependents * 75000
        is_disabled = input("您是否傷殘人士? (y/n): ").lower() == 'y'
        if is_disabled:
            total_allowances += 75000
        return income, total_deductions, total_allowances
    except ValueError:
        print("請輸入有效的數字！")
        return None, None, None
    except Exception as e:
        print(f"發生錯誤: {e}")
        return None, None, None

def register():
    print("\n=== 用戶註冊 ===")
    username = input("請輸入用戶名: ").strip()
    if username in users_db:
        print("用戶名已存在，請選擇其他用戶名！")
        return False
    password = input("請輸入密碼: ").strip()
    users_db[username] = {'password': password, 'tax_history': []}
    save_users_db(users_db)
    print("註冊成功！請登入使用。")
    return True

def login():
    print("\n=== 用戶登入 ===")
    username = input("請輸入用戶名: ").strip()
    password = input("請輸入密碼: ").strip()
    if username in users_db and users_db[username]['password'] == password:
        print(f"歡迎回來，{username}！")
        login_history.append(f"{username} logged in at {time.strftime('%Y-%m-%d %H:%M:%S')}")
        return username
    else:
        print("用戶名或密碼錯誤！")
        return None

def show_login_history():
    print("\n=== 最近登錄歷史 ===")
    if not login_history:
        print("尚無登錄記錄。")
    else:
        for entry in reversed(login_history):
            print(entry)

def show_tax_history(username):
    print(f"\n=== {username} 的稅務計算歷史 ===")
    tax_history = users_db[username].get('tax_history', [])
    if not tax_history:
        print("尚無稅務計算記錄。")
    else:
        for i, record in enumerate(tax_history, 1):
            print(f"\n記錄 {i} ({record['計算時間']}):")
            print("----------------")
            for key, value in record.items():
                if key != '計算時間':
                    print(f"{key}: ${value:,.2f}")
                else:
                    print(f"{key}: {value}")

def modify_user_credentials():
    print("\n=== 修改用戶憑證 ===")
    print("現有用戶:", ", ".join([u for u in users_db.keys() if u != "system"]))
    target_user = input("請輸入要修改的用戶名: ").strip()

    if target_user not in users_db or target_user == "system":
        print("用戶不存在或不能修改system用戶！")
        return

    new_username = input("輸入新用戶名 (留空保持不變): ").strip()
    new_password = input("輸入新密碼 (留空保持不變): ").strip()

    if new_username:
        if new_username in users_db:
            print("新用戶名已存在！")
            return
        users_db[new_username] = users_db.pop(target_user)
        if new_password:
            users_db[new_username]['password'] = new_password
        print(f"用戶名已從 {target_user} 修改為 {new_username}")
    elif new_password:
        users_db[target_user]['password'] = new_password
        print(f"用戶 {target_user} 的密碼已更新")
    else:
        print("未作出任何更改")

    save_users_db(users_db)

def show_system_statistics():
    print("\n=== 系統稅務統計 ===")
    all_records = []
    for username in users_db:
        if username != "system":
            all_records.extend(users_db[username].get('tax_history', []))

    if not all_records:
        print("尚無任何稅務記錄可供統計。")
        return

    total_records = len(all_records)
    total_income = sum(record['總收入'] for record in all_records)
    total_tax = sum(record['最終稅款(減免後)'] for record in all_records)
    total_deductions = sum(record['扣除額'] for record in all_records)
    total_allowances = sum(record['免稅額'] for record in all_records)
    avg_income = total_income / total_records
    avg_tax = total_tax / total_records
    tax_rates = [((record['最終稅款(減免後)'] / record['總收入']) * 100)
                for record in all_records if record['總收入'] > 0]
    avg_tax_rate = sum(tax_rates) / len(tax_rates) if tax_rates else 0

    print(f"總記錄數: {total_records}")
    print(f"總收入: ${total_income:,.2f}")
    print(f"總稅款: ${total_tax:,.2f}")
    print(f"總扣除額: ${total_deductions:,.2f}")
    print(f"總免稅額: ${total_allowances:,.2f}")
    print(f"平均年收入: ${avg_income:,.2f}")
    print(f"平均稅款: ${avg_tax:,.2f}")
    print(f"平均有效稅率: {avg_tax_rate:.2f}%")

    # 圖表1: 收入與稅款比例餅圖
    plt.figure(figsize=(8, 6))
    plt.pie([total_income - total_tax, total_tax],
            labels=['淨收入', '稅款'],
            autopct='%1.1f%%',
            colors=['#66b3ff', '#ff9999'])
    plt.title('收入與稅款比例\n顯示總收入中稅款所佔百分比')
    plt.show()

    # 圖表2: 稅率分佈餅圖
    if tax_rates:
        low_rate = sum(1 for rate in tax_rates if rate < 5)
        mid_rate = sum(1 for rate in tax_rates if 5 <= rate < 10)
        high_rate = sum(1 for rate in tax_rates if rate >= 10)

        plt.figure(figsize=(8, 6))
        plt.pie([low_rate, mid_rate, high_rate],
                labels=['0-5%', '5-10%', '10%+'],
                autopct='%1.1f%%',
                colors=['#99ff99', '#ffcc99', '#ff9999'])
        plt.title('有效稅率分佈\n顯示不同稅率區間的用戶分佈')
        plt.show()

    # 圖表3: 收入組成柱狀圖
    plt.figure(figsize=(10, 6))
    categories = ['總收入', '扣除額', '免稅額', '稅款']
    values = [total_income, total_deductions, total_allowances, total_tax]
    plt.bar(categories, values, color=['#66b3ff', '#ff9999', '#99ff99', '#ffcc99'])
    plt.title('收入組成分析\n比較總收入、扣除額、免稅額和稅款的金額')
    plt.ylabel('金額 (港元)')
    for i, v in enumerate(values):
        plt.text(i, v, f'${v:,.0f}', ha='center', va='bottom')
    plt.show()

def main():
    while True:
        print("\n=== 香港薪俸稅計算系統 ===")
        print("1. 註冊")
        print("2. 登入")
        print("3. 查看登錄歷史")
        print("4. 退出")
        choice = input("請選擇操作 (1-4): ").strip()

        if choice == '1':
            register()
        elif choice == '2':
            username = login()
            if username:
                if 'tax_history' not in users_db[username]:
                    users_db[username]['tax_history'] = []
                    save_users_db(users_db)

                while True:
                    print(f"\n=== 歡迎，{username} ===")
                    if username == "system" or "admin": #改main account
                        print("1. 查看系統統計")
                        print("2. 修改用戶憑證")
                        print("3. 查看用戶稅務歷史")
                        print("4. 登出")
                        sub_choice = input("請選擇操作 (1-4): ").strip()

                        if sub_choice == '1':
                            show_system_statistics()
                        elif sub_choice == '2':
                            modify_user_credentials()
                        elif sub_choice == '3':
                            target_user = input("請輸入要查看的用戶名: ").strip()
                            if target_user in users_db and target_user != "system":
                                show_tax_history(target_user)
                            else:
                                print("用戶不存在或不能查看system的歷史！")
                        elif sub_choice == '4':
                            print("已登出。")
                            break
                        else:
                            print("無效選項，請重新選擇！")
                    else:
                        print("1. 計算薪俸稅")
                        print("2. 查看稅務歷史")
                        print("3. 登出")
                        sub_choice = input("請選擇操作 (1-3): ").strip()

                        if sub_choice == '1':
                            income, deductions, allowances = get_user_input()
                            if income is not None:
                                result = calculate_hk_salary_tax(income, deductions, allowances)
                                print("\n計算結果:")
                                print("----------------")
                                for key, value in result.items():
                                    if key != '計算時間':
                                        print(f"{key}: ${value:,.2f}")
                                    else:
                                        print(f"{key}: {value}")
                                users_db[username]['tax_history'].append(result)
                                save_users_db(users_db)
                        elif sub_choice == '2':
                            show_tax_history(username)
                        elif sub_choice == '3':
                            print("已登出。")
                            break
                        else:
                            print("無效選項，請重新選擇！")
        elif choice == '3':
            show_login_history()
        elif choice == '4':
            print("感謝使用，再見！")
            break
        else:
            print("無效選項，請重新選擇！")

if __name__ == "__main__":
    main()


=== 香港薪俸稅計算系統 ===
1. 註冊
2. 登入
3. 查看登錄歷史
4. 退出


KeyboardInterrupt: Interrupted by user